In [1]:
import pandas as pd
import sklearn.metrics
import numpy as np
import os
import pickle as pickle

In [2]:
result = pd.read_csv('../data/dataset/test/dev_final.csv')


In [3]:
def klue_re_micro_f1(preds, labels):
    """KLUE-RE micro f1 (except no_relation)"""
    label_list = ['no_relation', 'org:top_members/employees', 'org:members',
       'org:product', 'per:title', 'org:alternate_names',
       'per:employee_of', 'org:place_of_headquarters', 'per:product',
       'org:number_of_employees/members', 'per:children',
       'per:place_of_residence', 'per:alternate_names',
       'per:other_family', 'per:colleagues', 'per:origin', 'per:siblings',
       'per:spouse', 'org:founded', 'org:political/religious_affiliation',
       'org:member_of', 'per:parents', 'org:dissolved',
       'per:schools_attended', 'per:date_of_death', 'per:date_of_birth',
       'per:place_of_birth', 'per:place_of_death', 'org:founded_by',
       'per:religion']
    no_relation_label_idx = label_list.index("no_relation")
    preds = label_to_num(preds.values)
    labels = label_to_num(labels.values)

    label_indices = list(range(len(label_list)))
    label_indices.remove(no_relation_label_idx)
    return sklearn.metrics.f1_score(labels, preds, average="micro", labels=label_indices) * 100.0


def label_to_num(label):
  num_label = []
  with open('../code/dict_label_to_num.pkl', 'rb') as f:
    dict_label_to_num = pickle.load(f)
  for v in label:
    num_label.append(dict_label_to_num[v])
  
  return num_label

In [4]:
# file_list = os.listdir('../code/prediction')
dir_name = './save_best'
dir_name = '../code/prediction'
file_list = os.listdir(f'{dir_name}')
dic={}
for file in file_list:
    if file[:3] == 'dev' or file == '.ipynb_checkpoints':
        continue
    # if not 'train_21' in file:
    #     continue
    x= pd.read_csv(f'{dir_name}/{file}')
    x= pd.concat([x,result],axis=1)
    dic[file]=klue_re_micro_f1(x.pred_label,x.label)
dic=dict(sorted(dic.items(), key=lambda x:-x[1]))
dic

{'train_19_roberta-large8379_checkpoint-6000_5_sub_57_4_5_19_roberta-large_lr_checkpoint-8000_21_roberta-large1_5_checkpoint-6000_1.csv': 75.70211423161881,
 'ens_train_19_roberta-large_lr_checkpoint-8000_train_21_roberta-large1_5_checkpoint-6000.csv': 75.45310015898251,
 'ens_train_21_roberta-large_train_19_roberta-large_lr.csv': 75.45310015898251,
 'train_19_roberta-large_lr_checkpoint-8000_21_roberta-large1_5_checkpoint-6000.csv': 75.45310015898251,
 'ens_train_21_roberta-large_train_21_seed_159.csv': 75.28179076043816,
 'train_21_seed_159.csv': 74.80190174326465,
 'train_19_roberta-large_lr_checkpoint-8000.csv': 74.58484896409932,
 'train_19_large8379_warnmup_checkpoint-6000.csv': 74.3671900032041,
 'train_21_seed_159_checkpoint-16000.csv': 74.34635496770224,
 'train_21_roberta-large copy.csv': 74.31796801505173,
 'train_21_roberta-large.csv': 74.31796801505173,
 'train_21_roberta-large1_5_checkpoint-6000.csv': 74.31796801505173,
 'train_19_roberta-large_lr_checkpoint-6000.csv': 74

In [7]:
import pickle as pickle
import os
import pandas as pd
import torch
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, Trainer, TrainingArguments, RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer, EarlyStoppingCallback

import random

def set_seed(seed:int = 42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    

In [9]:
set_seed(42)

In [10]:
print(torch.rand(1))

tensor([0.8823])
